In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import csv
import os
import pdfplumber
from PyPDF2 import PdfReader

In [2]:
folder_path = "E:\\Resume Dataset\\data\\data"


In [3]:
pdf_files = [file for file in os.listdir(folder_path) if file.endswith(".pdf")]


In [4]:
from PyPDF2 import PdfReader
import re

def extract_text_from_pdf(pdf_file_path):
    text = ""
    pdf_reader = PdfReader(pdf_file_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

def extract_category(text):
    category_pattern = r'Category: ([^\n]+)'
    match = re.search(category_pattern, text)
    return match.group(1) if match else None

def extract_skills(text):
    skills_pattern = r'Skills: ([^\n]+)'
    match = re.search(skills_pattern, text)
    return match.group(1) if match else None

def extract_education(text):
    education_pattern = r'Education: ([^\n]+)'
    match = re.search(education_pattern, text)
    return match.group(1) if match else None

# Specify the path to your PDF file
pdf_file_path = "E:\\Resume Dataset\\data\\data\\ACCOUNTANT\\10554236.pdf"

# Extract text from the PDF
text = extract_text_from_pdf(pdf_file_path)

# Extract key details
category = extract_category(text)
skills = extract_skills(text)
education = extract_education(text)

# Print the extracted details
print("Category (Job role):", category)
print("Skills:", skills)
print("Education (Degree, Institution):", education)


Category (Job role): None
Skills: None
Education (Degree, Institution): None


In [5]:
from datasets import load_dataset
import random

In [6]:
dataset = load_dataset("jacob-hugging-face/job-descriptions")

In [7]:
num_samples = random.randint(10, 15)
job_descriptions = random.sample(dataset["train"]["job_description"], num_samples)

In [8]:
for i, description in enumerate(job_descriptions):
    print(f"Job Description {i + 1}:\n")
    print(description)
    print("\n" + "=" * 50 + "\n")

Job Description 1:

senior project manager  education
job locations
us
posted date

 days ago
  pm
job id   of openings  category operation support type regular fulltime practice area education services

overview
about public consulting group
public consulting group inc pcg is a leading public sector solutions implementation and operations improvement firm that partners with health education and human services agencies to improve lives founded in  and headquartered in boston massachusetts pcg has over  professionals in more than  offices worldwideall committed to delivering solutions that change lives for the better the firm has four designated practice areas with extensive experience in all  states clients in six canadian provinces and a growing practice in europe often combining resources from two or more practice areas pcg offers clients a multidisciplinary approach to solve their challenges or pursue opportunities to learn more visit 
 
responsibilities
serve as a project manager a

In [9]:
import requests

# Specify the path to your custom certificate bundle
custom_cert_bundle_path = "C:\\Users\\hp\\Downloads\\cacert.pem"

# Make an HTTPS request using the custom certificate bundle
response = requests.get('https://example.com', verify=custom_cert_bundle_path)

# Check the response
if response.status_code == 200:
    print("Request successful")
else:
    print("Request failed")


Request successful


In [10]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)


In [12]:
# Example CV details and job descriptions (replace with your actual data)
cv_details = "I have a degree in Computer Science and am skilled in Python, machine learning, and data analysis."
job_descriptions = [
    "We are looking for a Data Scientist with a degree in Computer Science and expertise in Python and machine learning.",
    "Hiring a Python developer with strong skills in data analysis.",
    "Seeking a candidate with a background in Computer Science and experience in data analysis."
]

# Tokenize and encode CV details
cv_tokens = tokenizer.encode_plus(
    cv_details,
    add_special_tokens=True,
    max_length=128,
    pad_to_max_length=True,
    return_tensors="pt"
)

# Tokenize and encode job descriptions
job_desc_tokens = tokenizer.batch_encode_plus(
    job_descriptions,
    add_special_tokens=True,
    max_length=128,
    pad_to_max_length=True,
    return_tensors="pt",
    truncation=True
)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\hp\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [13]:
# Get embeddings for CV details
with torch.no_grad():
    cv_outputs = model(**cv_tokens)
    cv_embeddings = cv_outputs.last_hidden_state.mean(dim=1)

# Get embeddings for job descriptions
with torch.no_grad():
    job_desc_outputs = model(**job_desc_tokens)
    job_desc_embeddings = job_desc_outputs.last_hidden_state.mean(dim=1)


In [14]:
# Calculate cosine similarity
similarity_scores = cosine_similarity(cv_embeddings, job_desc_embeddings)

# Print the similarity scores
for i, score in enumerate(similarity_scores[0]):
    print(f"Similarity Score between CV and Job Description {i + 1}: {score}")


Similarity Score between CV and Job Description 1: 0.9215614795684814
Similarity Score between CV and Job Description 2: 0.8579851984977722
Similarity Score between CV and Job Description 3: 0.8638700246810913


In [15]:
#Embedding fn
def embedding(X_train):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    model = DistilBertModel.from_pretrained(model_name)
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    # X_train = job_description['job_description'].to_list()
    # print(type(X_train))

    res = tokenizer(X_train,padding=True, truncation=True, max_length=3, return_tensors="pt")
    # print(res)
    # len(res)

    with torch.no_grad():
        outputs = model(**res)
        # outputs = model(res['input_ids'])
        # model?
        JD_vects=outputs.last_hidden_state
    return JD_vects.reshape(1,-1)
    # return JD_vects[0].tolist()

In [16]:
#Cosine Similarity Function
def cosim(df1,df2):
    return cosine_similarity(df1,df2)

In [17]:
#import resume and jd csv filesimport resume and jd csv files
resume_data = pd.read_csv("E:\\Resume Dataset\\Resume\\Resume.csv",index_col=0)

In [18]:
# job_description = pd.read_csv('training_data.csv', chunksize=15)
job_description = pd.read_csv("D:\\training_data.csv")
job_description=job_description.head(15)

In [19]:
import re

# Define a function to clean text data
def clean(text):
    # Remove special characters, punctuation, and extra whitespace
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [20]:
#clean resume and jd text 
resume_data['Cleaned_Resume']=resume_data['Resume_html'].apply(lambda x:clean(x))
job_description['Cleaned_JD']=job_description['job_description'].apply(lambda x:clean(x))

In [21]:
#embed resume and jd text
resume_data['Cleaned_Resume_vector']=resume_data['Cleaned_Resume'].apply(lambda x:embedding(x))
job_description['Cleaned_JD_vector']=job_description['Cleaned_JD'].apply(lambda x:embedding(x))

In [22]:
#resume matching by cosine similarity
choice = int(input("Enter index of a job decription to choose from 0-14:\n"))

if choice <= 14 and choice > 0:
    # job_description['company_name']
    print(f"\nYour choice of JD is from {job_description[job_description['company_name'].index==choice]['company_name'].values[0]}\n")
    print(f"Job Description:\n\n {job_description[job_description['company_name'].index==choice]['job_description'].values[0]}\n")
    print(f"Position for which resumes are shortlisted: {job_description[job_description['company_name'].index==choice]['position_title'].values[0]}\n")

    resume_data['jd_vector_of_choice'] = resume_data['Cleaned_Resume_vector'].apply(lambda x: cosim(x,job_description['Cleaned_JD_vector'][choice].tolist()))
    print("Shortlisted resume File names are as below\n")
    # Sort resume_data by jd_vector_of_choice and get the top 5 rows
    top_resumes = resume_data.sort_values(by='jd_vector_of_choice', ascending=False).head(5)

    # Print the top 5 resumes
    print(top_resumes)

    #df=(resume_data.sort_values('jd_vector_of_choice', ascending=False)['File_name'].head())    
    #print(df)
    print()
else:
    print("Invalid choice try again")
    


Your choice of JD is from DesignUps

Job Description:

 designups is a nashville based design and interactive agency at designups we have a strong focus on elevating brands with our expertise in design and customer experience

are you a frontend developer with a serious interest in design and ui this could be a great fit if so this job is for someone in the greater nashville area the majority of the work can be done remotely but we do require some time in our east nashville office location hey its not too bad to walk to five points pizza or wildcow for lunch

this is a contract position to start with a competitive hourly rate we would like to find the right person to join our team for a longterm role

skills needed
strong attention to design details typography ui etc
translating designs responsively for multiple screen sizes
familiarity with bootstrap or other similar frameworks
wordpress knowledge and desire to learn is a major bonus

Position for which resumes are shortlisted: Front

In [27]:
try:
    for each in top_resumes:
        print('Category: '+resume_data[['Category','Resume_str']][resume_data['Resume_str']==each].values[0][0]+'\n')
        print('File Name: '+resume_data[['Category','File_name']][resume_data['File_name']==each].values[0][1]+'\n')
        print('Resume Content: \n\n'+resume_data['resume_text'][resume_data['File_name']==each].values[0]+'\n')
except IndexError:
    print('No resumes found that match the job description.')


No resumes found that match the job description.


In [23]:
import transformers
print(transformers.__version__)

4.33.1
